This notebook finds the RBE-weighted dose of the alpha particles and adds it to the RBE weighted dose of the protons.

In [1]:
#Install the Pydiocom Library
!pip install pydicom

     |████████████████████████████████| 1.9MB 2.9MB/s 


In [2]:
#Import the pydicom library to access the data from dicom files
import pydicom
import numpy as np

In [ ]:
def AlphaRBE(dose,LET,ab):
  '''Determines the RBE of a voxel using the QE method proposed by Mairani et al.
  Inputs are the voxel dose for one fraction, the LET of the alpha particles and 
  the alpha/beta ratio for the tissue. Returns the RBE of alpha particle.
  '''

  #Use parameters determined by Mairani et al
  k0 = 0.136938
  k1 = 0.00973154
  k2 = 0.0151998 

  #Find the value of the Quadratic Exponential (QE) LET function
  QE = (k1*(LET**2))*np.exp(-k2*LET)

  #Determine the maximum RBE
  max = 1 + (k0+(1/ab))*QE
  #Minimum RBE set to 1
  min = 1

  if dose == 0:
    RBE = min
  else:
    #Use linear quadratic model solution to find RBE
    RBE = (1/dose)*np.sqrt((1/4)*(ab**2) + max*ab*dose + min*(dose**2)) -ab/(2*dose)
  return RBE


In [ ]:
#Open the beam files
AlphaBase = pydicom.dcmread("/content/drive/MyDrive/AlphaDose/AlphaDose_Beam40_S1.dcm")
Alpha40 = pydicom.dcmread("/content/drive/MyDrive/AlphaDose/AlphaDose_Beam40_S1.dcm")
Alpha160 = pydicom.dcmread("/content/drive/MyDrive/AlphaDose/AlphaDose_Beam160_S1.dcm")
Alpha200 = pydicom.dcmread("/content/drive/MyDrive/AlphaDose/AlphaDose_Beam200_S1.dcm")
Alpha320 = pydicom.dcmread("/content/drive/MyDrive/AlphaDose/AlphaDose_Beam320_S1.dcm")

In [ ]:
#This script combines the four alpha particle dose files together

Alphaarr = AlphaBase.pixel_array
scale = 2*10**(-11)

#Iterate over each voxel
for z in range(AlphaBase.NumberOfFrames):
  for x in range(AlphaBase.Rows):
    for y in range(AlphaBase.Columns):
      Alphaarr[z][x][y] = (1/scale)*1000*(Alpha40.pixel_array[z][x][y]*Alpha40.DoseGridScaling +
                                           Alpha160.pixel_array[z][x][y]*Alpha160.DoseGridScaling + 
                                           Alpha200.pixel_array[z][x][y]*Alpha200.DoseGridScaling + 
                                           Alpha320.pixel_array[z][x][y]*Alpha320.DoseGridScaling)
#Save the output
AlphaBase.PixelData = Alphaarr.tobytes()     
AlphaBase.save_as("/content/drive/MyDrive/AlphaDose/AlphaOneFrac.dcm")

In [3]:
#This determine the RBE-weighted dose for the alpha particles and add it to the
#RBE weighted dose of the protons.

#Use the LET derived from energy spectrum
LET = 67.26585509840537
ab = 10.6

#Import alpha particle information
alpha = pydicom.dcmread("/content/drive/MyDrive/AlphaDose/AlphaOneFrac.dcm")
a_scale = 2*10**(-11)

#Import proton RBE weighted doses
proton = pydicom.dcmread("/content/drive/MyDrive/RBEscale/LETScale_CTV.dcm")
p_scale = 10**(-9)

base = pydicom.dcmread("/content/drive/MyDrive/RBEscale/LETScale_CTV.dcm")
array = base.pixel_array

#Iterate over each voxel
for z in range(base.NumberOfFrames):
  for x in range(base.Rows):
    for y in range(base.Columns):
      p_dose = proton.pixel_array[z][x][y]*p_scale
      a_dose = alpha.pixel_array[z][x][y]*a_scale
      a_dose_RBE = a_dose*AlphaRBE(a_dose,LET,ab)

      array[z][x][y] = (1/p_scale)*(a_dose_RBE+p_dose)

#Save the output
base.PixelData = array.tobytes() 
base.save_as("/content/drive/MyDrive/RBEscale/AlphaScale_CTV.dcm")

NameError: ignored